### Сравнение тарифов нового спонсора с тарифами Partner и P4

In [1]:
import pandas as pd
from pandas import Series, DataFrame
from os import path
import roamability as rb
from os.path import join
import numpy as np

downloads = r'c:\Users\balob\Downloads\DATA'

# Country;Roaming Partner;TADIG;MOC;MTC;SMS;DATA;Service
file_tariffs_new_sponsor = 'POL03_IOT_proposal_2019.csv'

dmi_allowed_list = 'DMI.dmi_allowed_list_export_Thu_May_30_2019.csv'
bss_usage = 'bss_usage_2019_05.csv'

In [2]:
# Получаем Data usage для Combined

# Like expression for Allowed List
df_al = pd.read_csv(join(downloads, dmi_allowed_list))
df_al = df_al[df_al.Sponsor.isin(['P4_Combined', 'Partner_Combined'])]
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*':'S1', r'[Pp]4[\s\w-]*':'S2'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)
df_al = df_al[['SPONSOR', 'TADIG']]

# Usage in Operators from BSS
df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['VPLMN','SPONSOR','DATA']
df_usage = df_usage.pivot_table(values='USAGE_QUANTITY', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]

df_usage.rename({'VPLMN':'TADIG', 'DATA':'DATA_VOL_MB'}, axis=1, inplace=True)
df_usage.DATA_VOL_MB = df_usage.DATA_VOL_MB/1024/1024
df_usage.columns.name=''

# Data usage in Operators from Combined
df_comb_usage = pd.merge(df_al, df_usage, how='left', on=['TADIG', 'SPONSOR'])
df_comb_usage.head(3)

,SPONSOR,TADIG,DATA_VOL_MB
0,S2,HRVT2,34535.650391
1,S2,MNEMT,15178.266602
2,S1,MKDCC,NaN


In [42]:
# Получаем данные тарифов нового Спонсора

df_new_sponsor = pd.read_csv(path.join(downloads, file_tariffs_new_sponsor), sep=';')
df_new_sponsor.drop(['Service'], axis=1, inplace=True)
df_new_sponsor.head(3)

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA
0,NaN,Monaco Maritime,AAMOM,0.5200,0.52,0.1300,1.9500
1,NaN,TMO M2M,AAZTD,0.0182,0.00,0.0059,0.0026
2,NaN,Vodafone M2M,AAZVF,0.0208,0.00,0.0020,0.0020


In [43]:
# Очистка данных от EUR, - и преобразование нескольких TADIG в один (Опционально)

# #####################################################
#Выбрать только первый TADIG из ячейки, если их там несколько через запятую
# #####################################################

# df_new_sponsor.TADIG = df_new_sponsor.TADIG.str.extract('([A-Z]+)', expand=True)

# #####################################################
# Очистить данные от EUR с конвертацией в USD
# #####################################################

#??????????????????????????????????????????????????????
# Как можно сделать тоже самое в цикле или с помощью функции?
#??????????????????????????????????????????????????????

# df_new_sponsor.loc[df_new_sponsor.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MOC'] =\
# df_new_sponsor.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

# df_new_sponsor.loc[df_new_sponsor.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MTC'] =\
# df_new_sponsor.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

# df_new_sponsor.loc[df_new_sponsor.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'SMS'] =\
# df_new_sponsor.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

# df_new_sponsor.loc[df_new_sponsor.DATA.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'DATA'] =\
# df_new_sponsor.DATA.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64')*1.2

#??????????????????????????????????????????????????????
# Почему .str.strip() обнулил данные для DNKTD?
#??????????????????????????????????????????????????????

#df_new_sponsor.MOC = df_new_sponsor.MOC.str.strip()
#df_new_sponsor.MTC = df_new_sponsor.MTC.str.strip()
#df_new_sponsor.SMS = df_new_sponsor.SMS.str.strip()
#df_new_sponsor.DATA = df_new_sponsor.DATA.str.strip()

# df_new_sponsor[['MOC','MTC','SMS','DATA']] = df_new_sponsor[['MOC','MTC','SMS','DATA']].replace(to_replace={r'-':0}, regex=True)

df_new_sponsor[['MOC','MTC','SMS','DATA']] = df_new_sponsor[['MOC','MTC','SMS','DATA']].astype('float64')

# #####################################################
# Конвертация EUR в USD
# #####################################################

df_new_sponsor[['MOC','MTC','SMS','DATA']] = df_new_sponsor[['MOC','MTC','SMS','DATA']] * 1.2

df_new_sponsor.head(3)

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA
0,NaN,Monaco Maritime,AAMOM,0.62400,0.624,0.15600,2.34000
1,NaN,TMO M2M,AAZTD,0.02184,0.000,0.00708,0.00312
2,NaN,Vodafone M2M,AAZVF,0.02496,0.000,0.00240,0.00240


In [44]:
df_new_sponsor.describe()

,Country,MOC,MTC,SMS,DATA
count,0.0,466.000000,466.000000,466.000000,466.000000
mean,NaN,0.278016,0.017225,0.039017,0.106256
std,NaN,0.339624,0.108951,0.053189,0.319580
min,NaN,0.015600,0.000000,0.001560,0.000960
25%,NaN,0.031200,0.000000,0.007800,0.007260
50%,NaN,0.218400,0.000000,0.031200,0.015600
75%,NaN,0.390000,0.000000,0.062400,0.062400
max,NaN,2.496000,1.560000,0.468000,3.120000


In [45]:
# Загружаем текущие тарифы Partner и P4

sql_srt=\
'''
SELECT
t.tadig_code AS TADIG,
IIF(CURRENCY_ID=1,MO_CALL_RATE,MO_CALL_RATE*1.2) MOC,
IIF(CURRENCY_ID=1,MT_CALL_RATE,MT_CALL_RATE*1.2) MTC,
IIF(CURRENCY_ID=1,MO_SMS_RATE,MO_SMS_RATE*1.2) SMS,
IIF(CURRENCY_ID=1,DATA_RATE,DATA_RATE*1.2) DATA,
r.SPONSOR_ID
FROM ROAMING_PLAN_RULES r
LEFT JOIN RDB_TADIG_CODES t ON r.network_id=t.network_id
WHERE roaming_plan_id IN (267,268)
AND r.is_discounted = 1
AND end_date IS NULL
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs = pd.read_sql_query(sql_srt, cnxn)

df_ocs_s1 = df_ocs.loc[df_ocs.SPONSOR_ID == 1, ['TADIG','MOC','MTC','SMS','DATA']]
df_ocs_s2 = df_ocs.loc[df_ocs.SPONSOR_ID == 2, ['TADIG','MOC','MTC','SMS','DATA']]
print('Partner current rates:')
display(df_ocs_s1.head(3))
print('\nP4 current rates:')
display(df_ocs_s2.head(3))

Partner current rates:


,TADIG,MOC,MTC,SMS,DATA
224,INDIB,0.10,0.0,0.0600,0.020
226,DEUE1,0.08,0.0,0.0200,0.013
228,ALBVF,0.03,0.0,0.0062,0.003



P4 current rates:


,TADIG,MOC,MTC,SMS,DATA
0,AIACW,0.960,0.0,0.096,0.09600
1,ATGCW,0.960,0.0,0.096,0.09600
2,AZEAC,0.504,0.0,0.084,0.01344


In [46]:
# Объединение новых тарифов с тарифами Партнер и P4

df_compare = pd.merge(df_new_sponsor, df_ocs_s1, how='outer', on='TADIG', suffixes=['', '_S1'])
df_compare = pd.merge(df_compare, df_ocs_s2, how='outer', on='TADIG', suffixes=['', '_S2'])
df_compare.head()

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA,MOC_S1,MTC_S1,SMS_S1,DATA_S1,MOC_S2,MTC_S2,SMS_S2,DATA_S2
0,NaN,Monaco Maritime,AAMOM,0.62400,0.624,0.15600,2.34000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,TMO M2M,AAZTD,0.02184,0.000,0.00708,0.00312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Vodafone M2M,AAZVF,0.02496,0.000,0.00240,0.00240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Digicel,ABWBC,0.26520,0.000,0.03120,0.06240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,MTN,AFGAR,0.46800,0.000,0.07800,0.15600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Add countries

sql_str='''SELECT DISTINCT c.country_name AS COUNTRY_NAME
,SUBSTRING(t.tadig_code,1,3) AS COUNTRY_PREF
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON c.country_id = n.country_id'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_country = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
df_country.head(3)

,COUNTRY_NAME,COUNTRY_PREF
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA


In [48]:
# Добавляем Usage и название стран

df_compare_usage = df_compare.copy()
df_compare_usage['COUNTRY_PREF'] = df_compare_usage.TADIG.str.slice(0,3)
df_compare_usage = pd.merge(df_compare_usage, df_comb_usage, how='left', on='TADIG')

df_compare_usage = pd.merge(df_compare_usage, df_country, how='left', on=['COUNTRY_PREF'])
df_compare_usage.head(3)

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA,MOC_S1,MTC_S1,SMS_S1,DATA_S1,MOC_S2,MTC_S2,SMS_S2,DATA_S2,COUNTRY_PREF,SPONSOR,DATA_VOL_MB,COUNTRY_NAME
0,NaN,Monaco Maritime,AAMOM,0.62400,0.624,0.15600,2.34000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAM,NaN,NaN,Iceland
1,NaN,TMO M2M,AAZTD,0.02184,0.000,0.00708,0.00312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAZ,NaN,NaN,France
2,NaN,TMO M2M,AAZTD,0.02184,0.000,0.00708,0.00312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAZ,NaN,NaN,Malta


In [49]:
df_compare_usage[df_compare_usage.COUNTRY_NAME.str.startswith('United Arab', na=False)]

,Country,Roaming Partner,TADIG,MOC,MTC,SMS,DATA,MOC_S1,MTC_S1,SMS_S1,DATA_S1,MOC_S2,MTC_S2,SMS_S2,DATA_S2,COUNTRY_PREF,SPONSOR,DATA_VOL_MB,COUNTRY_NAME
18,NaN,DU,AREDU,0.5148,0.0,0.06240,0.4212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARE,NaN,NaN,United Arab Emirates
20,NaN,Etisalat,ARETC,0.9828,0.0,0.06552,0.9360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARE,NaN,NaN,United Arab Emirates


In [50]:
# Выгружаем отчет в файл

columns = ['COUNTRY_NAME', 'Roaming Partner','TADIG','MOC','MOC_S1','MOC_S2'
           ,'MTC','MTC_S1','MTC_S2','SMS','SMS_S1','SMS_S2','DATA','DATA_S1','DATA_S2','DATA_VOL_MB']
df_compare_usage[columns].to_csv(path.join(downloads,'result_compare.csv'), index=False)

In [51]:
# Делаем группировку для Usage

cols = ['DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']
cols = ['DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']
df = df_compare_usage.groupby(['COUNTRY_NAME','COUNTRY_PREF'], as_index=False)[cols].\
agg({'DATA':[('', np.min), ('_PLMN', 'nunique')],
     'DATA_S1':[('', np.min), ('_PLMN', 'nunique')],
     'DATA_S2':[('', np.min), ('_PLMN', 'nunique')],
     'DATA_VOL_MB':[('', np.sum)], 'SPONSOR':[('', 'first')]})
df.columns = [''.join(col) for col in df.columns]

def usage_calc(x):
    data = x[['DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']]
    if x.SPONSOR == 'S1':
        x['COST_CURRENT'] = (x.DATA_VOL_MB) * x.DATA_S1
        if x.DATA < x.DATA_S1:
            x['COST_WITH_NEW'] = (x.DATA_VOL_MB) * x.DATA
        else:
            x['COST_WITH_NEW'] = x['COST_CURRENT']
    elif x.SPONSOR == 'S2':
        x['COST_CURRENT'] = (x.DATA_VOL_MB) * x.DATA_S2
        if x.DATA < x.DATA_S2:
            x['COST_WITH_NEW'] = (x.DATA_VOL_MB) * x.DATA
        else:
            x['COST_WITH_NEW'] = x['COST_CURRENT']
    return x
df_compare_usage_group = df.apply(usage_calc,axis=1)

df_compare_usage_group.head(3)

,COST_CURRENT,COST_WITH_NEW,COUNTRY_NAME,COUNTRY_PREF,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR
0,NaN,NaN,Afghanistan,AFG,0.1560,2,NaN,0,NaN,0,NaN,NaN
1,176.336896,176.336896,Albania,ALB,0.0156,2,0.003,1,0.0468,2,58778.96549,S1
2,NaN,NaN,Algeria,DZA,0.0390,2,NaN,0,NaN,0,NaN,NaN


In [52]:
cols = ['COUNTRY_NAME','DATA','DATA_PLMN','DATA_S1','DATA_S1_PLMN','DATA_S2','DATA_S2_PLMN','DATA_VOL_MB',
        'SPONSOR','COST_CURRENT','COST_WITH_NEW']
df_compare_usage_group = df_compare_usage_group[cols]
df_compare_usage_group.head(3)

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
0,Afghanistan,0.1560,2,NaN,0,NaN,0,NaN,NaN,NaN,NaN
1,Albania,0.0156,2,0.003,1,0.0468,2,58778.96549,S1,176.336896,176.336896
2,Algeria,0.0390,2,NaN,0,NaN,0,NaN,NaN,NaN,NaN


In [53]:
df_compare_usage_group[df_compare_usage_group.COUNTRY_NAME.str.startswith('Satellite', na=False)]

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
149,Satellite Networks,0.4212,2,NaN,0,NaN,0,NaN,NaN,NaN,NaN
150,Satellite Networks,0.0024,3,0.0037,2,0.0012,2,19588.448242,S2,23.506138,23.506138


### Analysis

In [54]:
# Новые страны

df_new = df_compare_usage_group[
    (df_compare_usage_group.DATA.notnull()) &
    (df_compare_usage_group.DATA_S1.isnull()) &
    (df_compare_usage_group.DATA_S2.isnull())]
df_new.head(3)

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
0,Afghanistan,0.156,2,NaN,0,NaN,0,NaN,NaN,NaN,NaN
2,Algeria,0.039,2,NaN,0,NaN,0,NaN,NaN,NaN,NaN
4,Angola,0.624,1,NaN,0,NaN,0,NaN,NaN,NaN,NaN


In [55]:
# Выгружаем отчет в файл

columns = ['COUNTRY_NAME', 'Roaming Partner','TADIG','MOC','MOC_S1','MOC_S2'
           ,'MTC','MTC_S1','MTC_S2','SMS','SMS_S1','SMS_S2','DATA','DATA_S1','DATA_S2','DATA_VOL_MB']

df_compare_usage[df_compare_usage.COUNTRY_NAME.isin(df_new.COUNTRY_NAME.tolist())][columns].\
to_csv(path.join(downloads,'result_new_countries.csv'), index=False)

In [56]:
# Дешевые тарифы

df_cheaper = df_compare_usage_group[
    df_compare_usage_group.COST_CURRENT > df_compare_usage_group.COST_WITH_NEW]
df_cheaper.head()

,COUNTRY_NAME,DATA,DATA_PLMN,DATA_S1,DATA_S1_PLMN,DATA_S2,DATA_S2_PLMN,DATA_VOL_MB,SPONSOR,COST_CURRENT,COST_WITH_NEW
39,Congo Dem. Rep,0.00780,3,0.0090,2,NaN,0,105.795972,S1,0.952164,0.825209
44,Cyprus,0.00156,2,0.0080,2,0.00252,2,119338.841003,S2,300.733879,186.168592
50,Dominican Republic,0.01560,2,0.0600,1,NaN,0,6915.751719,S1,414.945103,107.885727
58,France,0.00156,4,0.0055,1,0.00168,2,583142.126831,S2,979.678773,909.701718
59,French Guiana,0.00156,4,0.0055,1,0.00168,2,583142.126831,S2,979.678773,909.701718


In [57]:
# Выгружаем отчет в файл

columns = ['COUNTRY_NAME', 'Roaming Partner','TADIG','MOC','MOC_S1','MOC_S2'
           ,'MTC','MTC_S1','MTC_S2','SMS','SMS_S1','SMS_S2','DATA','DATA_S1','DATA_S2','DATA_VOL_MB','SPONSOR']

df_compare_usage[df_compare_usage.COUNTRY_NAME.isin(df_cheaper.COUNTRY_NAME.tolist())][columns].\
to_csv(path.join(downloads,'result_cheaper_countries.csv'), index=False)